In [40]:
from typing import *
from pathlib import Path
import numpy as np
import pandas as pd
import lightgbm as lgb
import pickle
from time import time

root = Path("/data/natsuki/dataset_atmaCup11")
"atma11sortingdate_j4nofreeze"
names = ["atma11simple_j4e5nofreeze", "atma11sortingdate_j4e5nofreeze", "atma11onehot_j4e5nofreeze", "atma11materialstechniques_j4e5nofreeze"]
epoch = 10
def aug(arr: List[np.ndarray]) -> np.ndarray:
    return np.mean(arr, axis=0).flatten()
def enc(t: int):
    return t

In [46]:
for suffix in ["3fold0", "3fold1", "3fold2"]:
    cv_list = list()
    for name in names:
        train_df = pd.read_csv(root/f"{suffix}_train.csv").set_index("object_id", drop=False)
        test_df = pd.read_csv(root/f"{suffix}_test.csv").set_index("object_id", drop=False)
        with open(root/"checkpoints"/f"{name}_{suffix}"/f"epoch{epoch}_{suffix}_train_features2.pkl", "rb") as f:
            train_dict = pickle.load(f)
        with open(root/"checkpoints"/f"{name}_{suffix}"/f"epoch{epoch}_{suffix}_test_features2.pkl", "rb") as f:
            test_dict = pickle.load(f)
        train_features = np.array([aug(train_dict[object_id]) for object_id in train_df["object_id"]])
        train_target = np.array([enc(train_df.loc[object_id]["target"]) for object_id in train_df["object_id"]])

array([3, 3, 1, ..., 2, 3, 1])